In [1]:
!pip install --upgrade pip
!pip install imutils
!pip install opencv-python

     |████████████████████████████████| 1.6MB 3.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from IPython.display import Image
from imutils import paths
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Для работы с изображениями, найдем их гистограммы — характеристики распределения интенсивности изображения. Для этого можно воспользоваться следующим методом.

In [3]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

Считаем изображения из корневой папки train и вычислим гистограмму каждого изображения. Кроме того, отделим метку каждого изображения. Обработка большого числа изображений в облачных сервисах занимает длительное время (1-5 минут).

In [4]:
imagePaths = sorted(list(paths.list_images(r'\Users\Nargo\Downloads\train')))
data = []
labels = []

for (i, imagePath) in enumerate(imagePaths):
    image = cv2.imread(imagePath, 1)
    label = imagePath.split(os.path.sep)[-1].split(".")[0]
    hist = extract_histogram(image)
    data.append(hist)
    labels.append(label)

In [5]:
type(imagePaths)

list

Преобразуем метки в удобный формат 0 и 1 и посмотрим, какой код и какая метка соответствует самой первой попавшейся нам картинке 

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(labels)
print(labels[0])
Image(filename=imagePaths[0])

Видно, что классом ноль у нас занумерованы коты (кошки).
Разделим выборку на тренировочную и тестовую в отношении 75/25 с параметром random_state = 51.

In [ ]:
trainData, testData, trainLabels, testLabels = train_test_split(np.array(data), labels, test_size=0.25, random_state=4)

Теперь обучим модель почти-разделяющий гиперплосоксти на наших данных, а также оценим ее на тестовых. Параметр C обратно пропорционален параметру модели, про который говорилось в лекции.

In [ ]:
model = LinearSVC(random_state = 4, C = 1.47)
model.fit(trainData, trainLabels)

In [ ]:
tetha = [152, 66, 123]
for coef in tetha:
    print("{0}\t{1:.2f}".format(coef, model.coef_[0][coef]))

In [ ]:
predictions = model.predict(testData)
print("F1_score\t{0:.3f}".format(f1_score(testLabels, predictions, average='macro')))

In [ ]:
test_list = ['cat.1020.jpg', 'cat.1010.jpg', 'dog.1046.jpg', 'dog.1013.jpg']
for image in test_list:
    path = r'C:\Users\Victor\Downloads\test\{0}'.format(image)
    singleImage = cv2.imread(path)
    histt = extract_histogram(singleImage)
    histt2 = histt.reshape(1, -1)
    prediction = model.predict(histt2)
    print(image, "\t", prediction)
    img = mpimg.imread(path)
    plt.imshow(img)
    plt.show()

Видно, что построенный классификатор успешно справился с заданием.